# Title 

Continuous Assessment 1 - Big Data & Adv. Data Analysis // MSc in Data Analytics Y1 S2 - <b>Student ID 2023063

## introduction

--

Data source: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop/data

## Practical

In [11]:
#Import PySparkSQL
from pyspark.sql import SparkSession        

# Create SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQL")
  .getOrCreate())

# Path to dataset
csv_file = "file:////home/hduser/Downloads/CA1/2020-Jan.csv"

In [15]:
# Read and create a temporary view
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("rawdata")

In [16]:
# Return data
data = spark.sql("""SELECT * FROM rawdata""")

In [19]:
# Visualise inferred schema
data.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [20]:
data.createOrReplaceTempView("event")

In [28]:
action = spark.sql("SELECT count(distinct event_type) FROM event GROUP BY 'product_id'")
action.show()

+--------------------------+
|count(DISTINCT event_type)|
+--------------------------+
|                         4|
+--------------------------+

